In [74]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, FloatType


In [75]:
def write_DB (parquet_DF, table):
    try:
        parquet_DF.write.jdbc(
            url=jdbc_url,
            table=table,
            mode="overwrite",
            properties=properties
        )
    except Exception as error:
        logger.error("Error ruta .yaml:" + str(error))

In [76]:

spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .config("spark.jars", "/home/illidan/proyecto_desde0/postgre/jars/postgresql-42.7.4.jar") \
    .getOrCreate()
)



spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))

errores_detectados = []


01:20:31.672 [Thread-4] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [77]:
#BD conection

try:
    jdbc_url = "jdbc:postgresql://localhost:5433/sparkdb_dev"
    properties = {
        "user": config[sparkdb_dev][user],
        "password": config[sparkdb_dev][password],
        "driver": "org.postgresql.Driver"
    }
except Exception as error:
    logger.error("Error conection BD:" + str(error))



01:20:31.687 [Thread-4] ERROR __main__ - Error conection BD:name 'sparkdb_dev' is not defined


In [78]:
#Read parquet file
try:
    read_avg_flight = config['Parquet_file']['avg_flight']
    read_airline_in_airport = config['Parquet_file']['airline_in_airport']
    read_flights_per_cancell = config['Parquet_file']['flights_per_cancell']
    read_airport_notin_thelist = config['Parquet_file']['airport_notin_thelist']
except Exception as error:
    logger.error("Error read parquet:" + str(error))


try:
    avg_flight = spark.read.parquet(read_avg_flight)
    airline_in_airport =  spark.read.parquet(read_airline_in_airport)
    flights_per_cancell = spark.read.parquet(read_flights_per_cancell)
    airport_notin_thelist = spark.read.parquet(read_airport_notin_thelist)
except Exception as error:
    logger.error("Error read parquet:" + str(error))

01:20:31.714 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.InMemoryFileIndex - It took 2 ms to list leaf files for 1 paths.
01:20:31.743 [Thread-4] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
01:20:31.744 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 29 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
01:20:31.744 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 29 (parquet at NativeMethodAccessorImpl.java:0)
01:20:31.744 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List()
01:20:31.744 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: List()
01:20:31.745 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting ResultStage 29 (MapPartitionsRDD[110] at parquet at NativeMethodAccessorImpl.java:0

In [ ]:
#write parquet file into DB

write_DB(avg_flight, "avg_flight")
write_DB(airline_in_airport,"airline_in_airport")
write_DB(flights_per_cancell,"flights_cancellation")
write_DB(airport_notin_thelist,"airport_notin_thelist")




01:20:32.279 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: 
01:20:32.280 [Thread-4] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: 
01:20:32.291 [Thread-4] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_47 stored as values in memory (estimated size 200.9 KiB, free 433.6 MiB)
01:20:32.309 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_46_piece0 on 172.23.57.81:44505 in memory (size: 37.2 KiB, free: 434.3 MiB)
01:20:32.311 [Thread-4] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_47_piece0 stored as bytes in memory (estimated size 34.9 KiB, free 433.7 MiB)
01:20:32.312 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Added broadcast_47_piece0 in memory on 172.23.57.81:44505 (size: 34.9 KiB, free: 434.3 MiB)
01:20:32.312 [Thread-4] INFO  org.apache.spark.SparkContext - Cr